# 🔄 Loop Workflows: Iterative Problem Refinement

**Use loop agents to continuously refine industry problems through critic feedback.**

This notebook demonstrates a **loop agent workflow** that iteratively refines industry problems. Instead of a one-shot analysis, a critic agent repeatedly reviews and eliminates weak or non-existent problems until only the most significant, location-specific challenges remain.

## Workflow:

1. **Problem Finder Agent** - Discovers problems in a specific industry within a city
2. **Critic Agent** (Loop) - Reviews problems and eliminates minimal/non-existent ones
3. **Final Output** - Clean list of real, substantial problems with one-sentence descriptions

**Why Loop?** Problems need multiple passes of validation and refinement to ensure accuracy. The critic agent continuously improves the list until high-quality problems remain.

**Use Case:** Market research, business opportunity analysis, location-based problem-solution matching.

---

## ⚙️ Section 1: Import Required Libraries

Set up ADK components for loop agent execution.

In [1]:
import os
from dotenv import load_dotenv
from google.adk.agents import Agent, LoopAgent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, FunctionTool
from google.genai import types

print("✅ All required libraries imported successfully.")

✅ All required libraries imported successfully.


---

## 🔑 Section 2: Configure Gemini API Authentication

Load your API key from the `.env` file.

In [2]:
# Load environment variables from .env file
load_dotenv()

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in .env file")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete. Ready to build loop agents!")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key setup complete. Ready to build loop agents!


---

## 🔍 Section 3: Define the Initial Problem Finder Agent

This agent discovers problems in a specific industry within a given city. It serves as the starting point for the loop.

**Input:** Career, industry, and city
**Output:** Comprehensive list of 10-15 industry problems in that location

In [3]:
initial_problem_finder_agent = Agent(
    name="InitialProblemFinderAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are an industry research specialist. Your job is to identify problems and challenges in a specific industry within a specific city.

Based on the career, industry, and city provided, research and identify:
1. 10-15 significant problems, challenges, or pain points the industry faces in that location
2. Include both well-known and emerging problems
3. Consider market conditions, competition, regulations, and local factors

Format your output as a numbered list with brief descriptions:
1. Problem Name - Brief description
2. Problem Name - Brief description
...

Output ONLY the numbered list. No introduction or additional text.""",
    tools=[google_search],
    output_key="current_problems",
)

print("✅ Initial Problem Finder Agent created successfully!")

✅ Initial Problem Finder Agent created successfully!


---

## 🎯 Section 4: Define the Critic Agent (Loop Core)

This agent reviews the current problem list and eliminates problems that are minimal, unlikely, or non-existent. It runs repeatedly in the loop until only high-quality problems remain.

**Input:** Current problem list
**Output:** Refined problem list with weak problems removed

In [4]:
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a critical analyst. Your job is to review industry problems and eliminate those that are minimal, unlikely, or don't actually exist.

You have received a list of industry problems:
{current_problems}

Your task:
1. Review EACH problem carefully
2. Eliminate problems that are:
   - Minimal or insignificant
   - Speculative or unlikely to exist
   - Already resolved or no longer relevant
   - Not specific to the industry/location
3. Keep only REAL, SUBSTANTIAL problems that genuinely affect the industry in that location
4. Output the refined list in the same numbered format

If NO problems meet the threshold of being real and substantial, output:
"ALL_PROBLEMS_REMOVED"

Otherwise, output ONLY the numbered list of remaining problems in this format:
1. Problem Name - Brief description
2. Problem Name - Brief description
...""",
    tools=[google_search],
    output_key="current_problems",
)

print("✅ Critic Agent created successfully!")

✅ Critic Agent created successfully!


---

## ✨ Section 5: Define the Final Output Agent

This agent takes the refined problem list and formats each problem as a single, concise sentence summary. It runs after the loop completes.

**Input:** Final refined problem list
**Output:** Clean list with one-sentence description per problem

In [5]:
final_output_agent = Agent(
    name="FinalOutputAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a concise summarizer. Your job is to format the final, refined problem list with one-sentence descriptions.

You have received the final refined problem list:
{current_problems}

Your task:
1. Take each problem
2. Rewrite it as a SINGLE, clear, concise sentence
3. Maintain the problem's essence but make it punchy and actionable
4. Format as:

1. [One sentence problem statement]
2. [One sentence problem statement]
3. [One sentence problem statement]
...

Output ONLY this formatted list. No introduction or additional commentary.""",
    output_key="final_clean_problems",
)

print("✅ Final Output Agent created successfully!")

✅ Final Output Agent created successfully!


---

## 🔄 Section 6: Create the Loop + Sequential Workflow

Now we combine the agents:
1. **Initial Stage** - Problem Finder generates initial problem list
2. **Loop Stage** - Critic agent reviews and eliminates weak problems (runs up to 3 iterations)
3. **Final Stage** - Output agent formats refined problems as one-sentence each

The loop provides iterative refinement until problems are high-quality.

In [6]:
# Create the Loop Agent - Critic runs repeatedly to refine problems
problem_refinement_loop = LoopAgent(
    name="ProblemRefinementLoop",
    sub_agents=[critic_agent],
    max_iterations=3,  # Max 3 refinement passes
)

# Create the overall workflow: Initial problems → Loop refinement → Final formatting
problem_pipeline = SequentialAgent(
    name="ProblemRefinementPipeline",
    sub_agents=[
        initial_problem_finder_agent,  # Step 1: Find initial problems
        problem_refinement_loop,       # Step 2: Refine via loop (up to 3 iterations)
        final_output_agent             # Step 3: Format final output
    ],
)

print("✅ Loop + Sequential Pipeline created!")
print("Pipeline flow: Problem Finder → [Loop: Critic x3] → Final Output Agent")

✅ Loop + Sequential Pipeline created!
Pipeline flow: Problem Finder → [Loop: Critic x3] → Final Output Agent


---

## 🚀 Section 7: Configure Your Inputs & Run the Pipeline

Enter your career, industry, and city. The pipeline will discover problems, refine them through iterations, and output a clean list.

👇 **Modify these three variables:**

In [7]:
# ===== SET YOUR INPUTS HERE =====
YOUR_CAREER = "Software Developer"
TARGET_INDUSTRY = "E-commerce"
CITY = "Lagos"

# ===== NO NEED TO MODIFY ANYTHING BELOW THIS LINE =====

# Initialize the runner
runner = InMemoryRunner(agent=problem_pipeline)

# Construct the prompt
prompt = f"""
Career: {YOUR_CAREER}
Industry: {TARGET_INDUSTRY}
City: {CITY}

Discover real, substantial problems facing the {TARGET_INDUSTRY} industry in {CITY}. These problems should be relevant for someone with a {YOUR_CAREER} background to potentially solve.

First, find 10-15 problems. Then, refine the list by removing minimal or non-existent problems through multiple passes. Finally, format the remaining problems as one-sentence descriptions.
"""

# Run the pipeline
print(f"🚀 Starting problem discovery and refinement")
print(f"Career: {YOUR_CAREER}")
print(f"Industry: {TARGET_INDUSTRY}")
print(f"City: {CITY}")
print(f"⏳ Discovering problems → Refining via critic loop → Formatting output...")
print()

response = await runner.run_debug(prompt)

print(f"\n✅ Pipeline completed!")
print(f"🎯 Final refined problem list generated above!")

🚀 Starting problem discovery and refinement
Career: Software Developer
Industry: E-commerce
City: Lagos
⏳ Discovering problems → Refining via critic loop → Formatting output...


 ### Created new session: debug_session_id

User > 
Career: Software Developer
Industry: E-commerce
City: Lagos

Discover real, substantial problems facing the E-commerce industry in Lagos. These problems should be relevant for someone with a Software Developer background to potentially solve.

First, find 10-15 problems. Then, refine the list by removing minimal or non-existent problems through multiple passes. Finally, format the remaining problems as one-sentence descriptions.

InitialProblemFinderAgent > 1. **Persistent Traffic Congestion** - Lagos's notorious traffic significantly impacts delivery times, increasing operational costs and negatively affecting customer satisfaction due to delays.
2. **Inconsistent Addressing System** - Vague or incomplete addresses make it challenging for delivery personnel 

---

## 📊 Understanding the Loop Workflow

### What Just Happened:

**Step 1 - Problem Discovery:**
1. ✅ **Problem Finder** identified 10-15 industry problems in the specified location

**Step 2 - Iterative Refinement (Loop):**
2. ✅ **Critic Agent Iteration 1** - Eliminated obviously weak or false problems
3. ✅ **Critic Agent Iteration 2** - Further refined, removing borderline cases
4. ✅ **Critic Agent Iteration 3** - Final pass for highest-quality problems only

**Step 3 - Final Formatting:**
5. ✅ **Final Output Agent** - Converted remaining problems to one-sentence descriptions

### The Loop Advantage:

🔄 **Single Pass:** Generic list of all potential problems  
🔄 **After Loop:** High-confidence, verified problems only  
🔄 **Quality Control:** Multiple passes ensure accuracy and relevance  

### Key Benefits:

✅ **Iterative refinement** - Problems get better with each pass  
✅ **Quality assurance** - Only real, substantial problems survive  
✅ **Location-specific** - Problems tailored to the specific city  
✅ **Actionable output** - One-sentence format for quick reference  

### Try Other Combinations:

Try different career/industry/city combinations:
- Full-Stack Developer / PropTech / Austin
- Data Scientist / Healthcare / Boston  
- Product Manager / FinTech / New York
- UX Designer / Ed-Tech / Seattle

---

## 🎓 Conclusion & Advanced Concepts

You've successfully built a **sophisticated loop agent workflow** that demonstrates:

### Core Concepts:

- **Loop Agents** for iterative, multi-pass processing
- **State persistence** - Problems improve through each iteration
- **Quality refinement** - Weak items eliminated automatically
- **Sequential + Loop combination** - Initial discovery → Iterative refinement → Final formatting
- **Location-specific analysis** - City context matters

### The Loop Architecture:

```
Problem Finder
      ↓
    [LOOP - 3 iterations]
    ├─ Iteration 1: Critic removes obvious weak problems
    ├─ Iteration 2: Critic removes borderline cases
    └─ Iteration 3: Critic validates remaining problems
      ↓
  Final Output
```

### Real-World Applications:

- Market opportunity validation
- Business problem discovery
- Location-specific market research
- Investment opportunity analysis
- Startup ideation and validation

### Customization Ideas:

- Add a **scoring agent** that rates problems by severity
- Add a **solution mapping agent** to suggest solutions
- Increase `max_iterations` for more refinement passes
- Add confidence levels to each problem
- Create a **competitor analysis loop**

### 📚 Learn More

- [ADK Loop Agents Documentation](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)
- [ADK Sequential Agents Documentation](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)
- [Combining Workflow Patterns](https://google.github.io/adk-docs/agents/)

Happy problem discovery and refinement! 🔄🎯